# Chapter 9 Exercises

3. Why is it generally preferable to use a logistic regression classifier rather than a classic perceptron (i.e., a single layer of threshold logic units trained using the perceptron training algorithm)? How can you tweak a perceptron to make it equivalent to a logistic regression classifier?


4. Why was the sigmoid activation function a key ingredient in training the first MLPs?

In [18]:
Because it helped randomize the weights in a way that was beneficial for escaping local minima.

SyntaxError: invalid syntax (1540740306.py, line 1)


5. Name three popular activation functions. Can you draw them?



6. Suppose you have an MLP composed of one input layer with 10 passthrough neurons, followed by one hidden layer with 50 artificial neurons, and finally one output layer with 3 artificial neurons. All artificial neurons use the ReLU activation function.

    a. What is the shape of the input matrix X?
        shape(X) = [,10]
    b. What are the shapes of the hidden layer’s weight matrix Wh and bias vector bh?
        shape(Wh) = [,50] bh
    c. What are the shapes of the output layer’s weight matrix Wo and bias vector bo?

    d. What is the shape of the network’s output matrix Y?

    e. Write the equation that computes the network’s output matrix Y as a function of X, Wh, bh, Wo, and bo.


7. How many neurons do you need in the output layer if you want to classify email into spam or ham? What activation function should you use in the output layer? If instead you want to tackle MNIST, how many neurons do you need in the output layer, and which activation function should you use? What about for getting your network to predict housing prices, as in Chapter 2?


8. What is backpropagation and how does it work? What is the difference between backpropagation and reverse-mode autodiff?


9. Can you list all the hyperparameters you can tweak in a basic MLP? If the MLP overfits the training data, how could you tweak these hyperparameters to try to solve the problem?


10. Train a deep MLP on the CoverType dataset. You can load it using `sklearn.datasets.fetch_covtype()`. See if you can get over 93% accuracy on the test set by fine-tuning the hyperparameters manually and/or using `RandomizedSearchCV`.

In [19]:
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline

covertype = fetch_covtype(as_frame=True)
x_train, x_test, y_train, y_test = train_test_split(
    covertype.data, covertype.target, random_state=42
    )

In [21]:
x_train.dtypes

Elevation                             float64
Aspect                                float64
Slope                                 float64
Horizontal_Distance_To_Hydrology      float64
Vertical_Distance_To_Hydrology        float64
Horizontal_Distance_To_Roadways       float64
Hillshade_9am                         float64
Hillshade_Noon                        float64
Hillshade_3pm                         float64
Horizontal_Distance_To_Fire_Points    float64
Wilderness_Area_0                     float64
Wilderness_Area_1                     float64
Wilderness_Area_2                     float64
Wilderness_Area_3                     float64
Soil_Type_0                           float64
Soil_Type_1                           float64
Soil_Type_2                           float64
Soil_Type_3                           float64
Soil_Type_4                           float64
Soil_Type_5                           float64
Soil_Type_6                           float64
Soil_Type_7                       

In [22]:
y_train.dtypes

dtype('int32')

In [23]:
y_train.value_counts()

Cover_Type
2    212525
1    158834
3     26845
7     15445
6     12994
5      7020
4      2096
Name: count, dtype: int64

Check if any columns have missing data where we would need the `SimpleImputer`

In [24]:
x_train.isnull().sum()

Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area_0                     0
Wilderness_Area_1                     0
Wilderness_Area_2                     0
Wilderness_Area_3                     0
Soil_Type_0                           0
Soil_Type_1                           0
Soil_Type_2                           0
Soil_Type_3                           0
Soil_Type_4                           0
Soil_Type_5                           0
Soil_Type_6                           0
Soil_Type_7                           0
Soil_Type_8                           0
Soil_Type_9                           0
Soil_Type_10                          0


First attempt using the MinMaxScaler and a [50,50,50] hidden layer shape.

We use the scaler to squish feature values into a range of 0 to 1. `scaled_value = (value - min) / (max - min)` since the ranges of our features are quite varied (Elevation is much larger numbers than slope for example) we are helping the model not give elevation more importantce than is due simply because the number is larger. 

In [ ]:
deep_mlp = MLPClassifier(hidden_layer_sizes=[50,50,50], early_stopping=True,
                       verbose=True, random_state=42)

In [25]:
pipeline = make_pipeline(MinMaxScaler(), deep_mlp)
pipeline.fit(x_train, y_train)
accuracy = pipeline.score(x_test, y_test)

Iteration 1, loss = 0.68989894
Validation score: 0.744446
Iteration 2, loss = 0.57965742
Validation score: 0.758261
Iteration 3, loss = 0.54685157
Validation score: 0.763631
Iteration 4, loss = 0.52175443
Validation score: 0.780269
Iteration 5, loss = 0.50272213
Validation score: 0.780843
Iteration 6, loss = 0.48942362
Validation score: 0.776184
Iteration 7, loss = 0.47798875
Validation score: 0.797457
Iteration 8, loss = 0.46866396
Validation score: 0.799431
Iteration 9, loss = 0.45927300
Validation score: 0.795002
Iteration 10, loss = 0.45102287
Validation score: 0.797021
Iteration 11, loss = 0.44236180
Validation score: 0.812787
Iteration 12, loss = 0.43483169
Validation score: 0.811387
Iteration 13, loss = 0.42832540
Validation score: 0.814990
Iteration 14, loss = 0.42277608
Validation score: 0.824904
Iteration 15, loss = 0.41616022
Validation score: 0.819373
Iteration 16, loss = 0.41099660
Validation score: 0.830021
Iteration 17, loss = 0.40626154
Validation score: 0.830274
Iterat

In [28]:
import ipywidgets as widgets
from IPython.display import display
import subprocess
import threading
import time

def get_temps():
    result = subprocess.run(['sensors'], capture_output=True, text=True)
    lines = result.stdout.split('\n')
    relevant = [l for l in lines if any(x in l for x in ['Tctl', 'edge:', 'Composite', 'fan1'])]
    return '\n'.join(relevant)

output = widgets.Output()
display(output)

def update_loop():
    while True:
        with output:
            output.clear_output(wait=True)
            print(get_temps())
        time.sleep(2)

thread = threading.Thread(target=update_loop, daemon=True)
thread.start()

Output()

In [27]:
print(accuracy)

0.8728287883899127


Now let's try with a different architecture, in the fashion MNST we used a larger first layer followed by a smaller second layer, let's try that here and see what we get.

In [29]:
deep_mlp = MLPClassifier(hidden_layer_sizes=[300,100], early_stopping=True,
                       verbose=True, random_state=42)
pipeline = make_pipeline(MinMaxScaler(), deep_mlp)
pipeline.fit(x_train, y_train)
accuracy = pipeline.score(x_test, y_test)

Iteration 1, loss = 0.63747068
Validation score: 0.755554
Iteration 2, loss = 0.54105040
Iteration 3, loss = 0.50411933
Validation score: 0.788186
Iteration 4, loss = 0.47742158
Validation score: 0.801175
Iteration 5, loss = 0.45515303
Validation score: 0.807623
Iteration 6, loss = 0.43690761
Validation score: 0.814026
Iteration 7, loss = 0.42099188
Validation score: 0.823664
Iteration 8, loss = 0.40723818
Validation score: 0.827497
Iteration 9, loss = 0.39655766
Iteration 10, loss = 0.38582343
Validation score: 0.833073
Iteration 11, loss = 0.37581527
Validation score: 0.834978
Iteration 12, loss = 0.36758265
Validation score: 0.840050
Iteration 13, loss = 0.36135320
Validation score: 0.828943
Iteration 14, loss = 0.35374555
Validation score: 0.851134


/home/smr/repos/hands-on-ml-notes/.venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:788: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
